### Fire Quasi-Perimeter Creation from VIIRS

Existing fire perimeter data available from CalFire and GeoMAC do not include a critical variable - the point of origin.  This is often and best determined in the field, based on multiple forensic clues.  This has been done for the 2018 Camp Fire, but not publically released.

There are several available satellite fire detection services, none of them perfect.  The NASA/NOAA VIIRs satellite provides 752m pixel samples every 6 minutes from geosynchronous orbit.  We used this data to generate an Omni points geodatabase with time stamps.

However, to make this compatible with later fire perimeter data, we need polygons.  Because of the course spatial resolution of the original data, and the 'spotting' behavior of the fire, we elected to do this by buffering the pixel centroids to reflect pixel spacing, and then merging buffered points per time step.

In [5]:
%run ../omnigeo.ipynb

Using pymapd version 0.7.1
OmniSci connection established on python global variable "con"
Using CPU memory (/dev/shm) for temp geo files
df: /dev/shm/csvs: No such file or directory


In [6]:
import geopandas as gpd
import pandas as pd
import os, glob

In [7]:
perimeters = 'fire_camp_perimeters'

In [8]:
q = f'select min(ST_XMin(omnisci_geo)) as xmin, '
q += f'max(ST_XMax(omnisci_geo)) as xmax, '
q += f'min(ST_YMin(omnisci_geo)) as ymin, '
q += f'max(ST_YMax(omnisci_geo)) as ymax '
q += f' from {perimeters}' 
q

'select min(ST_XMin(omnisci_geo)) as xmin, max(ST_XMax(omnisci_geo)) as xmax, min(ST_YMin(omnisci_geo)) as ymin, max(ST_YMax(omnisci_geo)) as ymax  from fire_camp_perimeters'

In [9]:
results = mapdql(q)

Executing query: select min(ST_XMin(omnisci_geo)) as xmin, max(ST_XMax(omnisci_geo)) as xmax, min(ST_YMin(omnisci_geo)) as ymin, max(ST_YMax(omnisci_geo)) as ymax  from fire_camp_perimeters


In [10]:
results.fetchall()

[(-121.777814599582, -121.352591766674, 39.5985802028322, 39.8978104581581)]

### Extract Point of Origin

In [62]:
all_points_of_origin = 'fire_2018_sit_rep_pts_geomac'

In [63]:
def getColumnNamesTypes(table_name):
    table_meta = con.get_table_details(all_points_of_origin)
    col_names = []
    col_types = []
    for meta in table_meta:
        col_names.append(meta[0])
        col_types.append(meta[1])
    return(col_names, col_types)   


In [64]:
def selectFirePOI(fire_name):
    q = f'SELECT * FROM {all_points_of_origin} '
    q += f"WHERE incidentna = '{fire_name.upper()}'"
    results = mapdql(q)
    df = pd.DataFrame(results.fetchall())
    cols, types = getColumnNamesTypes(all_points_of_origin)
    df.columns = cols
    return(df)

In [65]:
df = selectFirePOI('WOOLSEY')
df

Executing query: SELECT * FROM fire_2018_sit_rep_pts_geomac WHERE incidentna = 'WOOLSEY'


,latitude,longitude,acres,gacc,hotlink,state,status,firecause,reportdate,percentcon,...,pooownerun,incidentty,mergeid,containmen,pooprotect,complexnam,perimexist,irwinmodif,invalid,omnisci_geo
0,34.24,-118.7,96949.0,OSCC,http://www.nifc.gov/fireInfo/nfn.htm,CA,F,Unknown,1970-01-01,100,...,CAVNC,WF,None,None,Local Agency,None,Y,1970-01-01,None,POINT (-118.699999953946 34.2399999984726)


### Convert from Pandas to Geodataframe

In [66]:
import geopandas as gpd
from shapely.geometry import Point
from shapely import wkt
import shutil

In [67]:
def pandas2geo(df):
    # convert pandas dataframe to geodataframe using the shapely geometry column
    df['shapely_point'] = df['omnisci_geo'].apply(wkt.loads)

    gdf = gpd.GeoDataFrame(df, geometry='shapely_point')
    return(gdf)

In [68]:
origin_gdf = pandas2geo(df)

#### Export Simple Origin Point for GIS Munging

Turns out to be amazingly hard... if any datetime fields exist, geopandas and fiona will refuse to export to csv, shapefile, etc.  In theory, geopackage should work, but doesn't in practice.

In [69]:
def _get_col_dtype(col):
        """
        Infer datatype of a pandas column, process only if the column dtype is object. 
        input:   col: a pandas Series representing a df column. 
        """

        if col.dtype =="object":

            # try numeric
            try:
                col_new = pd.to_datetime(col.dropna().unique())
                return col_new.dtype
            except:
                try:
                    col_new = pd.to_numeric(col.dropna().unique())
                    return col_new.dtype
                except:
                    try:
                        col_new = pd.to_timedelta(col.dropna().unique())
                        return col_new.dtype
                    except:
                        return "object"

        else:
            return col.dtype

In [70]:
def getDateTimeFields(df):
    dtf = []
    for fieldname in df.columns:
        #print(fieldname, _get_col_dtype(df[fieldname]))
        if _get_col_dtype(df[fieldname]) == 'datetime64[ns]':
            dtf.append(fieldname)
    return(dtf)

In [71]:
getDateTimeFields(origin_gdf)

['reportdate',
 'firediscov',
 'complexpar',
 'fireoutdat',
 'datecurren',
 'poolandown',
 'mergeid',
 'containmen',
 'complexnam',
 'irwinmodif',
 'invalid']

Note: some old fields are mischaraterized

In [74]:
def force2str(df):
    for col in getDateTimeFields(df):
        print(f'Forcing column {col} to string for export')
        df[col].apply(str)
    return(df)

In [75]:
origin_gdf = force2str(origin_gdf)

Forcing column reportdate to string for export
Forcing column firediscov to string for export
Forcing column complexpar to string for export
Forcing column fireoutdat to string for export
Forcing column datecurren to string for export
Forcing column poolandown to string for export
Forcing column mergeid to string for export
Forcing column containmen to string for export
Forcing column complexnam to string for export
Forcing column irwinmodif to string for export
Forcing column invalid to string for export


In [82]:
for o in origin_gdf.columns:
    print(f'{o} is type {origin_gdf[o].dtype}')

latitude is type float64
longitude is type float64
acres is type float64
gacc is type object
hotlink is type object
state is type object
status is type object
firecause is type object
reportdate is type object
percentcon is type int64
uniquefire is type object
firediscov is type object
complexpar is type object
poorespons is type object
incidentna is type object
irwinid is type object
fireoutdat is type object
datecurren is type object
fireyear is type int64
poolandown is type object
pooownerun is type object
incidentty is type object
mergeid is type object
containmen is type object
pooprotect is type object
complexnam is type object
perimexist is type object
irwinmodif is type object
invalid is type object
omnisci_geo is type object
shapely_point is type object


In [83]:
origin_gdf.drop(getDateTimeFields(origin_gdf),inplace=True, axis=1)

In [113]:
def poiShape(origin_gdf):
    fire_name = origin_gdf['incidentna'][0].lower()
    shapedir = f'data/origins/{fire_name}'
    if not os.path.exists(shapedir):
        !mkdir {shapedir}
    outfile = f'data/origins/fire_{fire_name}_approx_origin_point.shp'
    origin_gdf.to_file(outfile)
    zip_name = f'fire_{fire_name}_origin_point'
    shutil.make_archive(f'{zip_name}', 'zip', shapedir)
    print(f'File saved to: {zip_name}.zip')

In [114]:
poiShape(origin_gdf)

File saved to: fire_woolsey_origin_point.zip


#### Buffer Point to Pseudo-Perimeter

In [131]:
# before doing buffering, we need to project into a planar coordinate system
origin_gdf.crs = {'init' :'epsg:4326'}
try:
    origin_gdf = origin_gdf.to_crs({'init':'EPSG:3857'})
except:
    print('Problem projecting point into planar coordinates for buffering')

In [132]:
origin_gdf.crs

{'init': 'EPSG:3857'}

In [133]:
origin_gdf.type

0    Point
dtype: object

In [134]:
# assign to new column to keep others....
origin_gdf['geometry'] = origin_gdf.geometry.buffer(752/2)

In [135]:
origin_gdf.head()

,latitude,longitude,acres,gacc,hotlink,state,status,firecause,reportdate,percentcon,...,mergeid,containmen,pooprotect,complexnam,perimexist,irwinmodif,invalid,omnisci_geo,shapely_point,geometry
0,39.82,-121.44,153336.0,ONCC,http://www.nifc.gov/fireInfo/nfn.htm,CA,F,Unknown,1970-01-01,100,...,None,1970-01-01,CDF,None,Y,1970-01-01,None,POINT (-121.439999929368 39.8199999936949),POINT (-13518638.95407242 4839819.542282384),POLYGON ((-13518262.95407242 4839819.542282384...


In [136]:
try:
    # set_geometry('geometry', crs=df.crs)
    origin_gdf.set_geometry('geometry', crs=origin_gdf.crs)
except:
    print('Problem setting new geometry column')

In [137]:
origin_gdf.drop(['shapely_point','omnisci_geo'],inplace=True,axis=1)

In [138]:
origin_gdf.head()

,latitude,longitude,acres,gacc,hotlink,state,status,firecause,reportdate,percentcon,...,pooownerun,incidentty,mergeid,containmen,pooprotect,complexnam,perimexist,irwinmodif,invalid,geometry
0,39.82,-121.44,153336.0,ONCC,http://www.nifc.gov/fireInfo/nfn.htm,CA,F,Unknown,1970-01-01,100,...,CAPNF,WF,None,1970-01-01,CDF,None,Y,1970-01-01,None,POLYGON ((-13518262.95407242 4839819.542282384...


In [216]:
origin_gdf.geometry

0    POLYGON ((-13518262.95407242 4839819.542282384...
Name: geometry, dtype: object

In [217]:
origin_gdf.crs

{'init': 'EPSG:3857'}

In [235]:
# not sure why this should be necessary, except somehow we now have a series
# instead of a geodataframe (because just one row?)
o2 = gpd.GeoDataFrame(origin_gdf)

In [236]:
o2.crs = 'epsg:3857'

Weird and annoying issue: geopandas will not correctly export datetime fields.<br>Based on SO, need to manually define schema

In [237]:
origin_gdf.columns

Index(['latitude', 'longitude', 'acres', 'gacc', 'hotlink', 'state', 'status',
       'firecause', 'reportdate', 'percentcon', 'uniquefire', 'firediscov',
       'complexpar', 'poorespons', 'incidentna', 'irwinid', 'fireoutdat',
       'datecurren', 'fireyear', 'poolandown', 'pooownerun', 'incidentty',
       'mergeid', 'containmen', 'pooprotect', 'complexnam', 'perimexist',
       'irwinmodif', 'invalid', 'geometry'],
      dtype='object')

Yipes, burried deep inside geopandas is a way to infer schema...

In [261]:
schema = gpd.io.file.infer_schema(o2)
print(schema)

{'geometry': 'Polygon', 'properties': OrderedDict([('latitude', 'float'), ('longitude', 'float'), ('acres', 'float'), ('gacc', 'str'), ('hotlink', 'str'), ('state', 'str'), ('status', 'str'), ('firecause', 'str'), ('percentcon', 'int'), ('uniquefire', 'str'), ('poorespons', 'str'), ('incidentna', 'str'), ('irwinid', 'str'), ('fireyear', 'int'), ('poolandown', 'str'), ('pooownerun', 'str'), ('incidentty', 'str'), ('pooprotect', 'str'), ('perimexist', 'str')])}


In [262]:
o2.columns

Index(['latitude', 'longitude', 'acres', 'gacc', 'hotlink', 'state', 'status',
       'firecause', 'percentcon', 'uniquefire', 'poorespons', 'incidentna',
       'irwinid', 'fireyear', 'poolandown', 'pooownerun', 'incidentty',
       'pooprotect', 'perimexist', 'geometry'],
      dtype='object')

In [263]:
def redatifySchema(schema):
    fixed = {}
    for key, value in schema['properties'].items():
        # do something with value
        if key in getDateTimeFields(o2):
            print(f'Writing new key for {key}')
            schema['properties'][key] = 'datetime'
    return(schema)       

In [264]:
redatifySchema(schema)

{'geometry': 'Polygon',
 'properties': OrderedDict([('latitude', 'float'),
              ('longitude', 'float'),
              ('acres', 'float'),
              ('gacc', 'str'),
              ('hotlink', 'str'),
              ('state', 'str'),
              ('status', 'str'),
              ('firecause', 'str'),
              ('percentcon', 'int'),
              ('uniquefire', 'str'),
              ('poorespons', 'str'),
              ('incidentna', 'str'),
              ('irwinid', 'str'),
              ('fireyear', 'int'),
              ('poolandown', 'str'),
              ('pooownerun', 'str'),
              ('incidentty', 'str'),
              ('pooprotect', 'str'),
              ('perimexist', 'str')])}

In [265]:
fiona.__version__

'1.8.4'

In [266]:
!gdalinfo --version

GDAL 2.4.0, released 2018/12/14


In [267]:
import traceback
import sys

if 0:
    # now back to geo, since wkt loader requires WGS84
    try:
        origin_gdf = o2.to_crs(epsg=4326)
    except:
        print('Problem back projecting to WGS84')
        exc_type, exc_value, exc_tb = sys.exc_info()
        traceback.print_exception(exc_type, exc_value, exc_tb)    

else:
    print('Back projection not working, using ogr')
    driver = 'ESRI Shapefile'
    ext = 'shp'
    planar_file = f'origin_poly_3857.{ext}'
    o2.to_file(planar_file, schema=schema, driver=f'{driver}')
    geo_file = planar_file.replace(f'3857.{ext}',f'geo.{ext}')
    !ogr2ogr {geo_file} {planar_file}
origin_gdf.crs

Back projection not working, using ogr


{'init': 'EPSG:3857'}

In [270]:
!ls *.shp

camp_fire_origin_perimeter.shp	origin_poly_3857.shp  origin_poly_geo.shp


OK, now have a geo file on disk as shape, without any datetime columns<br>
Now want to load that, and force back at least the single datetime representing fire perimeter time

In [271]:
origin_gdf = gpd.read_file(geo_file)

In [272]:
# follow schemas of geoMAC fire perimeters for easier merging later
origin_gdf['perDatTime'] = str('2018-11-08T06:30PCT')

In [273]:
origin_gdf.head()

,latitude,longitude,acres,gacc,hotlink,state,status,firecause,percentcon,uniquefire,...,incidentna,irwinid,fireyear,poolandown,pooownerun,incidentty,pooprotect,perimexist,geometry,perDatTime
0,39.82,-121.44,153336.0,ONCC,http://www.nifc.gov/fireInfo/nfn.htm,CA,F,Unknown,100,2018-CABTU-016737,...,CAMP,{75E64DB8-9B75-4A68-BEDD-67CC62658E38},2018,USFS,CAPNF,WF,CDF,Y,POLYGON ((-13518262.95407242 4839819.542282384...,2018-11-08T06:30PCT


In [274]:
outfile = 'camp_fire_origin_perimeter.csv'
if os.path.exists(outfile):
    !rm {outfile}

In [275]:
origin_gdf.to_csv(outfile, index=False)

In [276]:
!head {outfile}

latitude,longitude,acres,gacc,hotlink,state,status,firecause,percentcon,uniquefire,poorespons,incidentna,irwinid,fireyear,poolandown,pooownerun,incidentty,pooprotect,perimexist,geometry,perDatTime
39.82,-121.44,153336.0,ONCC,http://www.nifc.gov/fireInfo/nfn.htm,CA,F,Unknown,100,2018-CABTU-016737,CABTU,CAMP,{75E64DB8-9B75-4A68-BEDD-67CC62658E38},2018,USFS,CAPNF,WF,CDF,Y,"POLYGON ((-13518262.95407242 4839819.542282384, -13518264.76461519 4839782.68783762, -13518270.17880699 4839746.188321305, -13518279.14450619 4839710.395243736, -13518291.5753682 4839675.653311814, -13518307.35167703 4839642.297109338, -13518326.3214982 4839610.647874769, -13518348.30214196 4839581.010407538, -13518373.0819227 4839553.670132658, -13518400.42219758 4839528.890351919, -13518430.05966481 4839506.909708158, -13518461.70889938 4839487.939886989, -13518495.06510185 4839472.16357816, -13518529.80703378 4839459.732716149, -13518565.60011134 4839450.767016952, -13518602.09962766 4839445.352825155, -13518638.9540

Weird: despite projecting back to WGS84 without error message, values in the geometry column are still webb mercator 3857....

In [29]:


ddl = '('
for col_name, col_type in zip(col_names, col_types):
    if col_type == 'STR':
        col_type = 'TEXT'
    if col_type == 'POINT':
        col_type = "POLYGON"
    ddl += f'{col_name} {col_type}, '
ddl += 'perDatTime TIMESTAMP'
ddl = ddl + ')'
ddl

'(latitude DOUBLE, longitude DOUBLE, acres DOUBLE, gacc TEXT, hotlink TEXT, state TEXT, status TEXT, firecause TEXT, reportdate DATE, percentcon INT, uniquefire TEXT, firediscov DATE, complexpar TEXT, poorespons TEXT, incidentna TEXT, irwinid TEXT, fireoutdat DATE, datecurren DATE, fireyear INT, poolandown TEXT, pooownerun TEXT, incidentty TEXT, mergeid TEXT, containmen DATE, pooprotect TEXT, complexnam TEXT, perimexist TEXT, irwinmodif DATE, invalid TEXT, omnisci_geo GEOMETRY(POLYGON,3857), perDatTime TIMESTAMP)'

In [30]:
table_name = 'fire_camp_approx_origin'
mapdql(f"DROP TABLE IF EXISTS {table_name}")

Executing query: DROP TABLE IF EXISTS fire_camp_approx_origin


In [31]:
mapdql(f"CREATE TABLE {table_name} {ddl}")

Executing query: CREATE TABLE fire_camp_approx_origin (latitude DOUBLE, longitude DOUBLE, acres DOUBLE, gacc TEXT, hotlink TEXT, state TEXT, status TEXT, firecause TEXT, reportdate DATE, percentcon INT, uniquefire TEXT, firediscov DATE, complexpar TEXT, poorespons TEXT, incidentna TEXT, irwinid TEXT, fireoutdat DATE, datecurren DATE, fireyear INT, poolandown TEXT, pooownerun TEXT, incidentty TEXT, mergeid TEXT, containmen DATE, pooprotect TEXT, complexnam TEXT, perimexist TEXT, irwinmodif DATE, invalid TEXT, omnisci_geo GEOMETRY(POLYGON,3857), perDatTime TIMESTAMP)


In [32]:
cwd_list = !pwd
cwd = cwd_list[0]
fullpath = os.path.join(cwd, outfile)

In [33]:
result = mapdql(f"COPY {table_name} FROM '{fullpath}' ")
result

Executing query: COPY fire_camp_approx_origin FROM '/home/mapdadmin/demo/fire/camp_fire_origin_perimeter.csv' 


In [34]:
result.fetchone()

('Loaded: 1 recs, Rejected: 0 recs in 1.647000 secs',)

In [210]:
table_name in con.get_tables()

True